In [15]:
import cv2
import os
from keras_preprocessing.image import ImageDataGenerator
import numpy as np

In [19]:
PATH_TO_ORIGINAL_IMAGE_DIRECTORY = "../CharactersPlate"
PATH_TO_AAUGMENTED_IMAGE_DIRECTORY = "../AugmentedCharactersPlate"
THRESHOLD = 127
WINDOW_SIZE = 1000

In [20]:
def show_image_on_window(image: np.array, window_name: str = "Image") -> None:
    scale = max(image.shape[:2]) / WINDOW_SIZE
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, int(
        image.shape[1] // scale), int(image.shape[0] // scale))
    cv2.imshow(window_name, image)
    cv2.waitKey(0)
    cv2.destroyWindow(window_name)

In [25]:
def adapt_image(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image_binary = cv2.threshold(image, THRESHOLD, 255, cv2.THRESH_BINARY)
    image_binary = image_binary/255
    return image_binary.flatten()    

In [26]:
def load_images(dir_path):
        images = []
        for file_name in os.listdir(dir_path):
                original_image = cv2.imread(dir_path + "/" + file_name)
                flat_image = adapt_image(original_image)
                images.append(flat_image)
        return np.array(images) 

In [27]:
X = load_images(PATH_TO_ORIGINAL_IMAGE_DIRECTORY)

(792,)


DATA AUGMENTATION

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

datagen.fit(X)